In [41]:
from dotenv import load_dotenv
import re
import json
import os
import redis
from tmdbv3api import TMDb, TV, Season, Episode
import xml.etree.ElementTree as ET

load_dotenv()

True

In [4]:
r = redis.Redis(host='localhost', port=6379, decode_responses=True)

Test Key:

In [7]:
# Example: Set and get a key
r.set('key', 'value')

True

In [4]:
print(r.get('key'))  # b'value'

value


In [10]:
# Add the media file metadata from the NFO file
metadata = {
    'plot': 'In 2071, roughly fifty years after an accident with a hyperspace gateway made the Earth almost uninhabitable, humanity has colonized most of the rocky planets and moons of the Solar System. Amid a rising crime rate, the Inter Solar System Police (ISSP) set up a legalized contract system, in which registered bounty hunters (also referred to as "Cowboys") chase criminals and bring them in alive in return for a reward.',
    'outline': 'In 2071, roughly fifty years after an accident with a hyperspace gateway made the Earth almost uninhabitable, humanity has colonized most of the rocky planets and moons of the Solar System. Amid a rising crime rate, the Inter Solar System Police (ISSP) set up a legalized contract system, in which registered bounty hunters (also referred to as "Cowboys") chase criminals and bring them in alive in return for a reward.',
    'lockdata': 'true',
    'dateadded': '2022-10-19 19:46:52',
    'title': 'Cowboy Bebop',
    'originaltitle': 'カウボーイビバップ',
    'writer': 'Hajime Yatate',
    'credits': 'Hajime Yatate',
    'trailer': 'plugin://plugin.video.youtube/?action=play_video&videoid=RI08P5SaJNU',
    'rating': '8.9',
    'year': '1998',
    'mpaa': 'TV-14',
    'imdb_id': 'tt0213338',
    'tmdbid': '30991',
    'premiered': '1998-04-02',
    'releasedate': '1998-04-02',
    'enddate': '1999-04-23',
    'runtime': '25',
    'genre': 'Western',
    'studio': 'WOWOW Prime',
    'tag': 'space western',
    'tvdbid': '76885',
    'art': '',
    'actor': '',
    'id': '76885',
    'episodeguide': '',
    'season': '-1',
    'episode': '-1',
    'status': 'Ended'
}

# Use hset with mapping to add the metadata
r.hset('media:cowboy_bebop', mapping=metadata)

0

In [5]:
# Retrieve metadata
print(r.hgetall('media:cowboy_bebop'))

{'status': 'Ended', 'writer': 'Hajime Yatate', 'year': '1998', 'studio': 'WOWOW Prime', 'mpaa': 'TV-14', 'plot': 'In 2071, roughly fifty years after an accident with a hyperspace gateway made the Earth almost uninhabitable, humanity has colonized most of the rocky planets and moons of the Solar System. Amid a rising crime rate, the Inter Solar System Police (ISSP) set up a legalized contract system, in which registered bounty hunters (also referred to as "Cowboys") chase criminals and bring them in alive in return for a reward.', 'imdb_id': 'tt0213338', 'releasedate': '1998-04-02', 'lockdata': 'true', 'id': '76885', 'credits': 'Hajime Yatate', 'enddate': '1999-04-23', 'episode': '-1', 'premiered': '1998-04-02', 'originaltitle': 'カウボーイビバップ', 'dateadded': '2022-10-19 19:46:52', 'season': '-1', 'runtime': '25', 'rating': '8.9', 'actor': '', 'outline': 'In 2071, roughly fifty years after an accident with a hyperspace gateway made the Earth almost uninhabitable, humanity has colonized most 

Script to process nfo

In [ ]:
# Load and parse the NFO file
tree = ET.parse('tvshow.nfo')
root = tree.getroot()

# Extract metadata from the NFO file
metadata = {}
for child in root:
    metadata[child.tag] = child.text

metadata

Functions

In [94]:
def parse_filename(filename):
    # Regular expressions for season and episode
    patterns = [
        r'S(?P<season>\d{2})E(?P<episode>\d{2})',  # Matches S01E01 format
        r's(?P<season>\d{2})e(?P<episode>\d{2})',   # Matches s01e01 format
        r's(?P<season>\d{4})e(?P<episode>\d{2})',   # Matches s01e01 format
        r'(?P<season>\d)X(?P<episode>\d{2})',       # Matches 1X01 format
        r'(?P<season>\d{2})x(?P<episode>\d{2})',    # Matches 01x01 format
        r'(?P<season>\d{2})\.(?P<episode>\d{2})',   # Matches 01.01 format
        r'(?P<season>\d{1})E(?P<episode>\d{2})',    # Matches 1E01 format
        r'(?P<season>\d{1})X(?P<episode>\d{2})',    # Matches 1X01 format with single-digit season
        r'(?P<season>\d{1})-(?P<episode>\d{1,2})'   # Matches 1-1 format
    ]
    
    # Check for season and episode numbers
    for pattern in patterns:
        match = re.search(pattern, filename, re.IGNORECASE)
        if match:
            return match.groupdict()
    
    # Regular expression for episode only
    episode_patterns = [
        r'E(?P<episode>\d{2})',  # Matches E01 format
        r'e(?P<episode>\d{2})',  # Matches e01 format
        r'(?P<episode>\d{2})'    # Matches 01 format
    ]
    
    # Check for episode numbers only
    for pattern in episode_patterns:
        match = re.search(pattern, filename, re.IGNORECASE)
        if match:
            return {'season': 1, 'episode': match.group('episode')}
    
    return None

def parse_filenames(filenames):
    parsed_info = []
    for filename in filenames:
        info = parse_filename(filename)
        if info:
            parsed_info.append({'filename': filename, 'season': info.get('season'), 'episode': info.get('episode')})
        else:
            parsed_info.append({'filename': filename, 'season': None, 'episode': None})
    return parsed_info

def clean_tv_show_title(title):
    title = re.sub(r'(x264|x265|HVEC).*', '', title, flags=re.IGNORECASE)
    title = re.sub(r'\[.*?\]', '', title)
    title = re.sub(r'\(.*?\)', '', title)
    title = re.sub(r'\.(avi|mkv|mp4|WEBRip|web|AMZN|NF|DEFLATE|DDP5\.1|720p|1080p|Complete|Original|S\d{2}|E\d{2})', '', title, flags=re.IGNORECASE)
    title = title.replace('.', ' ')
    title = re.sub(r'\s+', ' ', title).strip()
    title = title.title()
    return title

# def get_first_element(variable):
#     if isinstance(variable, list):
#         return variable[0] if variable else None
#     return variable

def is_video_file(filename):
    video_extensions = ['.mp4', '.mkv', '.avi', '.mov']
    return os.path.splitext(filename)[1].lower() in video_extensions

def index_files_by_show(root_path):
    show_index = {}
    exclude_file_types = json.loads(os.getenv('IGNORED_FILE_TYPES', '[]'))
    excluded_files = json.loads(os.getenv('IGNORED_FILES', '[]'))
    
    def dfs(path, show_name):
        try:
            contents = os.listdir(path)
            for item in contents:
                full_path = os.path.join(path, item)
                if os.path.isdir(full_path):
                    dfs(full_path, show_name)
                elif os.path.isfile(full_path) and is_video_file(item):
                    if (os.path.splitext(item)[1] not in exclude_file_types) and (item not in excluded_files):
                        show_index[show_name].append(full_path)
        except PermissionError:
            print(f"Permission denied: {path}")
    
    for show in os.listdir(root_path):
        show_path = os.path.join(root_path, show)
        if os.path.isdir(show_path):
            show_index[show] = []
            dfs(show_path, show)
    
    return show_index

def add_show_metadata(metadata, metadata_type):
    match metadata_type:
        case 'show':
            r.hset(f'show:{metadata["title"]}', mapping=metadata)
        case 'season':
            r.hset(f'show:{metadata["title"]}:season:{metadata["season"]}', mapping=metadata)
        case 'episode':
            r.hset(f'show:{metadata["title"]}:season:{metadata["season"]}:episode:{metadata["episode"]}', mapping=metadata)
        case _:
            print('Invalid metadata type')
            
def fetch__and_add_metadata(show_index):
    tv = TV()
    season = Season()
    episode = Episode()

    for show in show_index:
        search_results = tv.search(show)
        results = list(search_results.results)
        if not(results):
            cleaned_title = clean_tv_show_title(show)
            search_results = tv.search(cleaned_title)
            results = list(search_results.results)
        show_id = results[0].id
        show_metadata = tv.details(show_id)
        show_metadata.path = os.path.join(os.getenv("MEDIA_PATH"), show)
        # add_show_metadata(show_metadata, 'show')

        file_paths = show_index[show]
        for file_path in file_paths:
            file = os.path.basename(file_path)
            file = parse_filename(file)
            seasonNum = file.get('season')
            episodeNum = file.get('episode')

            season_metadata = season.details(show_id, seasonNum)
            # add_show_metadata(season_metadata, 'season')
            
            episode_metadata = episode.details(show_id, seasonNum, episodeNum)
            episode_metadata.path = file_path
            # add_show_metadata(episode_metadata, 'episode')


In [95]:
file_path = os.getenv('MEDIA_PATH')
show_index = index_files_by_show(file_path)
# result = fetch_metadata(show_index)
# with open("burner.txt", "w") as f:
#   print(result, file=f)

In [96]:
api_key = os.getenv('TMDB_API_KEY')
tmdb = TMDb()
tmdb.api_key = api_key
show_title = 'Cowboy Bebop'
seasonNum = 1
episodeNum = 1
tv = TV()

results = tv.search(show_title)
show_id = results[0].id
show_details = tv.details(show_id)

# metadata = fetch_metadata(show_title, seasonNum, episodeNum)
# print(metadata)

In [41]:
# for title in result[1]:
#     clean_title = re.sub(r'\.S\d{2}.*', '', title).replace('.', ' ')
#     print(clean_title)
#     print()